In [23]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

In [24]:
PM10 = pd.read_csv('./data/PM10-2023.csv', sep=';')
NO2 = pd.read_csv('./data/NO2-2023.csv', sep=';')
traffic = pd.DataFrame()
for file in os.listdir('./data/natezenie_ruchu_2023'):
    data = pd.read_csv('./data/natezenie_ruchu_2023/' + file, sep=';')
    traffic = pd.concat([traffic, data])

In [25]:
PM10 = PM10.drop_duplicates()
traffic = traffic.drop_duplicates()
NO2 = NO2.drop_duplicates()

In [26]:
traffic.columns = ['Traffic_station_id', 'date', 'hour', 'quality', 'num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed', 'num_trucks', 'avg_truck_speed']
df = pd.read_excel('./data/Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
# a = []
# for i, item in enumerate(df['DET_ID15']):
#     if item in traffic['station_id'].unique():
#         a.append([df['x'][i], df['y'][i]])

In [27]:
traffic_id = [100101010031590, 100101010031287]
traffic['date'] = pd.to_datetime(traffic['date'], format="%d.%m.%Y")
traffic['hour'] = pd.to_timedelta(traffic['hour'], unit='h')
traffic['datetime'] = traffic['date'] + traffic['hour']
traffic = traffic.drop(columns=['date', 'hour'])
traffic = traffic[traffic['Traffic_station_id'].isin(traffic_id)]

In [28]:
PM10['Date'] = pd.to_datetime(PM10['Date'], format="%Y-%m-%d")
PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
PM10['Time'][PM10['Time'] == '24:00'] = '00:00'
PM10['Time'] = pd.to_timedelta(PM10['Time'].str.strip("'") + ':00')
PM10['datetime'] = PM10['Date'] + PM10['Time']
PM10 = PM10.drop(columns=['Date', 'Time', "State / Measuring network", "Station setting","Station type","Pollutant","Time scope", "Unit", "Station name"])

C:\Users\ASamek.ERP\AppData\Local\Temp\ipykernel_23556\72315281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
C:\Users\ASamek.ERP\AppData\Local\Temp\ipykernel_23556\72315281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Time'][PM10['Time'] == '24:00'] = '00:00'


In [29]:
print(len(traffic))
print(len(PM10))

13432
17518


In [30]:
merged_df = pd.merge(traffic, PM10, left_on='datetime', right_on='datetime', how='inner')

In [31]:
pivot_df = merged_df.pivot_table(index=['datetime'],
                          columns=['Station code', 'Traffic_station_id'],
                          values=['num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed',
                                  'num_trucks', 'avg_truck_speed', 'Measure value','quality'],
                          aggfunc='first')
pivot_df.columns = ['{}_{}_{}'.format(col[1], col[0], i) for i, col in enumerate(pivot_df.columns.values)]
pivot_df.reset_index(inplace=True)
pivot_df.columns

Index(['datetime', 'DEBE068_Measure value_0', 'DEBE068_Measure value_1',
       'DEBE125_Measure value_2', 'DEBE125_Measure value_3',
       'DEBE068_avg_car_speed_4', 'DEBE068_avg_car_speed_5',
       'DEBE125_avg_car_speed_6', 'DEBE125_avg_car_speed_7',
       'DEBE068_avg_speed_8', 'DEBE068_avg_speed_9', 'DEBE125_avg_speed_10',
       'DEBE125_avg_speed_11', 'DEBE068_avg_truck_speed_12',
       'DEBE068_avg_truck_speed_13', 'DEBE125_avg_truck_speed_14',
       'DEBE125_avg_truck_speed_15', 'DEBE068_num_cars_16',
       'DEBE068_num_cars_17', 'DEBE125_num_cars_18', 'DEBE125_num_cars_19',
       'DEBE068_num_trucks_20', 'DEBE068_num_trucks_21',
       'DEBE125_num_trucks_22', 'DEBE125_num_trucks_23',
       'DEBE068_num_vehicles_24', 'DEBE068_num_vehicles_25',
       'DEBE125_num_vehicles_26', 'DEBE125_num_vehicles_27',
       'DEBE068_quality_28', 'DEBE068_quality_29', 'DEBE125_quality_30',
       'DEBE125_quality_31'],
      dtype='object')

In [32]:
# columns_to_drop = list(range(1, len(pivot_df.columns), 2))
# pivot_df = pivot_df.drop(pivot_df.columns[columns_to_drop], axis=1)
pivot_df.to_csv("pivot.csv", index=False)

In [33]:
columns_to_drop

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]